In [1]:


# Uninstall mkl for faster neural-network training time
# Upgrade pip to ensure the latest package versions are available
!pip install mxnet --upgrade
!pip install dask --upgrade
!pip install -U "mxnet<2.0.0"
!pip install autogluon
!pip install -U ipykernel

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
np.random.seed(111)
from __future__ import print_function


import mxnet as mx
import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon
from mxnet.gluon.data.vision import transforms
from autogluon import TextPrediction as task

from src import load_text, lemma_text

ctx = mx.gpu() if mx.context.num_gpus() else mx.cpu()

Requirement already up-to-date: mxnet in /usr/local/lib/python3.6/dist-packages (1.7.0.post2)
Requirement already up-to-date: dask in /usr/local/lib/python3.6/dist-packages (2021.2.0)
Requirement already up-to-date: mxnet<2.0.0 in /usr/local/lib/python3.6/dist-packages (1.7.0.post2)
Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.4.3)


/content/src.py:17: DeprecationWarning: invalid escape sequence \;
  df = re.split('\;|\n|\.|\?|\!', df)
/content/src.py:21: DeprecationWarning: invalid escape sequence \;
  df.Text = df.Text.str.split(pat='\;|\n|\.|\?|\!', expand=False)
/content/src.py:97: DeprecationWarning: invalid escape sequence \w
  return series.str.replace('www.','',regex=False).replace('\w*\.\w{2,}', value = "*url", regex=True)
/content/src.py:107: DeprecationWarning: invalid escape sequence \w
  return series.str.replace('[A-Z]\w+',repl,regex=True)


# New Section

In [2]:
df = load_text(sentences=True).dropna()


NumExpr defaulting to 2 threads.


In [3]:
from src import lemma_text
copy_df = df.copy()
copy_df['lemmas'] = lemma_text(df['Text'])
copy_df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


LookupError: ignored

In [ ]:
randindx = np.random.permutation(np.arange(len(df)))
split = int(len(randindx)*.8)
train_df = df.iloc[randindx[:split]]
test_df = df.iloc[randindx[split:]]
errors = test_df.copy()


In [ ]:
train_df['StrGrade'] = train_df['Grade'].astype(str)

In [ ]:
from autogluon import TextPrediction as task

predictor = task.fit(train_data=task.Dataset(train_df[['StrGrade','Grade']]),
                     label='Grade', verbosity=3
                    )

ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: botocore 1.19.59 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
I

In [ ]:

predictor = task.fit(train_df[['Text','Grade']], label='Grade',
                     seed=111
                    )
print(predictor.evaluate(test_df[['Text','Grade']], metrics=['mae']))
yhat = predictor.predict({'Text':[text for text in test_df.Text]})
errors['Text Prediction'] = yhat
errors

In [ ]:
predictor2 = task.fit(train_df[['grammarized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor2.evaluate(test_df[['grammarized','Grade']], metrics=['mae']))
yhat = predictor2.predict({'grammarized':[text for text in test_df.grammarized]})
errors['Grammarized Prediction'] = yhat
errors

In [ ]:
predictor3 = task.fit(train_df[['lemmatized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor3.evaluate(test_df[['lemmatized','Grade']], metrics=['mae']))
yhat = predictor3.predict({'lemmatized':[text for text in test_df.Text]})
errors['Lemmatized Prediction'] = yhat
errors

In [ ]:
errors = errors.iloc[:,[1,2,3,0,4,5]]
errors

In [ ]:
predictor.export('TextModel')

In [ ]:
net.export("lenet", epoch=1)


In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    deserialized_net = gluon.nn.SymbolBlock.imports("lenet-symbol.json", ['data'], "lenet-0001.params", ctx=ctx)
